<a href="https://colab.research.google.com/github/salami96/python/blob/main/projeto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio:

Você faz parte da equipe de Analytics de uma grande marca de vestuário com mais de 25 lojas espalhadas em Shoppings de todo o Brasil.

Toda semana você precisa enviar para a diretoria um ranking atualizado com as 25 lojas contendo 3 informações:
- Faturamento de cada Loja
- Quantidade de Produtos Vendidos de cada Loja
- Ticket Médio dos Produto de cada Loja

Além disso, cada loja tem 1 gerente que precisa receber o resumo das informações da loja dele. Por isso, cada gerente deve receber no e-mail:
- Faturamento da sua loja
- Quantidade de Produtos Vendidos da sua loja
- Ticket Médio dos Produto da sua Loja

Esse relatório é sempre enviado como um resumo de todos os dados disponíveis no ano.

Passo a passo de construção do código:

- Passo 1: Importar arquivo XLSX
- Passo 2: Calcular o faturamento de cada loja
- Passo 3: Calcular a quant. de produtos vendidos de cada loja
- Passo 4: Calcular ticket médio dos produtos de cada loja
- Passo 5: Enviar e-mail para diretoria
- Passo 6: Enviar e-mail para cada loja

Passo 1: Importar base de dados

In [ ]:
import pandas as pd

tabela_vendas = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Vendas.xlsx')
display(tabela_vendas)

Passo 2: Calcular faturamento de cada loja

In [ ]:
tabela_faturamento = tabela_vendas[['ID Loja','Valor Final']].groupby('ID Loja').sum()
tabela_faturamento = tabela_faturamento.sort_values(by = 'Valor Final', ascending = False)

display(tabela_faturamento)

Passo 3: Quantidade de Produtos vendido em cada loja

In [ ]:
tabela_quant = tabela_vendas[['ID Loja','Quantidade']].groupby('ID Loja').sum()
tabela_quant = tabela_quant.sort_values(by = 'Quantidade', ascending = False)

display(tabela_quant)

Passo 4: Ticket médio dos produtos de cada loja

In [ ]:
tabela_ticket = (tabela_faturamento['Valor Final'] / tabela_quant['Quantidade']).to_frame('Ticket Médio')
tabela_ticket = tabela_ticket.sort_values(by = 'Ticket Médio', ascending = False)

display(tabela_ticket)

Função de enviar e-mail:

In [ ]:
def enviar_email(nome_da_loja, tabela):
  import smtplib
  import email.message

  server = smtplib.SMTP('smtp.gmail.com:587')  
  corpo_email = f'''
  <p>Prezados,</p>
  <p>Segue relatório de vendas</p>
  <table>
  {tabela.to_html()}
  </table>
  <p>Qualquer dúvida estou a disposição</p>
  '''
    
  msg = email.message.Message()
  msg['Subject'] = f'Relatório de Vendas - {nome_da_loja}'
    
  # Fazer antes (apenas na 1ª vez): Ativar Aplicativos não Seguros.
  # Gerenciar Conta Google -> Segurança -> Aplicativos não Seguros -> Habilitar
  # Caso mesmo assim dê o erro: smtplib.SMTPAuthenticationError: (534,
  # Você faz o login no seu e-mail e depois entra em: https://accounts.google.com/DisplayUnlockCaptcha
  msg['From'] = 'gabriel.zanatto2@gmail.com'
  msg['To'] = 'gabriel.zanatto2@gmail.com'
  password = "salami1996"
  msg.add_header('Content-Type', 'text/html')
  msg.set_payload(corpo_email)
    
  s = smtplib.SMTP('smtp.gmail.com: 587')
  s.starttls()
  # Login Credentials for sending the mail
  s.login(msg['From'], password)
  s.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
  print('Email enviado')

Passo 5: Enviar e-mail para diretoria

In [ ]:
tabela_completa = tabela_faturamento.join(tabela_quant).join(tabela_ticket)
enviar_email('Diretoria', tabela_completa)

Email enviado


Passo 6: Enviar e-mail para cada loja

In [ ]:
lista_lojas = tabela_vendas['ID Loja'].unique()

for loja in lista_lojas:
  tabela_loja = tabela_vendas.loc[tabela_vendas['ID Loja'] == loja,['ID Loja', 'Quantidade', 'Valor Final']]
  tabela_loja = tabela_loja.groupby('ID Loja').sum()
  tabela_loja['Ticket Médio'] = (tabela_loja['Valor Final'] / tabela_loja['Quantidade'])
  enviar_email(loja, tabela_loja)